# Make a Video Mosaic

This notebook creates a mosaic for a specifc day.
The mosaic is mad of all image, with overlay of their ID.

In [1]:
from __future__ import division
%matplotlib inline
import cv2
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm_notebook

In [2]:
#
# Load data and print Head
#
data_folder = '/VISL2_net/amitibo/experiment_forecast_2017_08/'
csv_filename = 'all_days.csv'

df = pd.read_csv(
    os.path.join(data_folder, csv_filename),
    index_col=[0],
    header=[0, 1],
    skiprows=[2],
    parse_dates=[0],
    infer_datetime_format=True
)
df.head()

cam_110                                   \
                    altitude   exposure   latitude  longitude   
2017-08-13 03:01:00     97.0  399981.84  32.712653  35.107428   
2017-08-13 03:02:00     97.0  399981.84  32.712653  35.107428   
2017-08-13 03:03:00     97.0  399981.84  32.712653  35.107428   
2017-08-13 03:04:00     97.0  146445.20  32.712653  35.107428   
2017-08-13 03:05:00     97.0   70437.52  32.712653  35.107428   

                                                                        \
                                                                  path   
2017-08-13 03:01:00  /VISL2_net/amitibo/experiment_forecast_2017_08...   
2017-08-13 03:02:00  /VISL2_net/amitibo/experiment_forecast_2017_08...   
2017-08-13 03:03:00  /VISL2_net/amitibo/experiment_forecast_2017_08...   
2017-08-13 03:04:00  /VISL2_net/amitibo/experiment_forecast_2017_08...   
2017-08-13 03:05:00  /VISL2_net/amitibo/experiment_forecast_2017_08...   

                                   cam_111                                   \
                       serial_num altitude   exposure   latitude  longitude   
2017-08-13 03:01:00  4.102820e+09     15.0  399981.84  32.824437  35.046428   
2017-08-13 03:02:00  4.102820e+09     15.0  399981.84  32.824437  35.046428   
2017-08-13 03:03:00  4.102820e+09     15.0  399981.84  32.824437  35.046428   
2017-08-13 03:04:00  4.102820e+09     15.0  159011.60  32.824437  35.046428   
2017-08-13 03:05:00  4.102820e+09     15.0   73331.60  32.824437  35.046428   

                       ...        cam_123                        \
                       ...       exposure   latitude  longitude   
2017-08-13 03:01:00    ...      399981.84  32.775776  35.024963   
2017-08-13 03:02:00    ...      399981.84  32.775776  35.024963   
2017-08-13 03:03:00    ...      399981.84  32.775776  35.024963   
2017-08-13 03:04:00    ...      195263.76  32.775776  35.024963   
2017-08-13 03:05:00    ...       80262.16  32.775776  35.024963   

                                                                        \
                                                                  path   
2017-08-13 03:01:00  /VISL2_net/amitibo/experiment_forecast_2017_08...   
2017-08-13 03:02:00  /VISL2_net/amitibo/experiment_forecast_2017_08...   
2017-08-13 03:03:00  /VISL2_net/amitibo/experiment_forecast_2017_08...   
2017-08-13 03:04:00  /VISL2_net/amitibo/experiment_forecast_2017_08...   
2017-08-13 03:05:00  /VISL2_net/amitibo/experiment_forecast_2017_08...   

                                   cam_219                                     
                       serial_num altitude latitude longitude path serial_num  
2017-08-13 03:01:00  4.103099e+09      NaN      NaN       NaN  NaN        NaN  
2017-08-13 03:02:00  4.103099e+09      NaN      NaN       NaN  NaN        NaN  
2017-08-13 03:03:00  4.103099e+09      NaN      NaN       NaN  NaN        NaN  
2017-08-13 03:04:00  4.103099e+09      NaN      NaN       NaN  NaN        NaN  
2017-08-13 03:05:00  4.103099e+09      NaN      NaN       NaN  NaN        NaN  

[5 rows x 83 columns]

## Select "Good" Data

Good data is:

1. No Nan values for camera 219 (label camera)
2. At least 8 cameras.

In [3]:
idx = pd.IndexSlice

df = df.loc[:, idx[:, 'path']]
df.columns = df.columns.droplevel(1)

df = df[~pd.isnull(df["cam_219"])]
df = df.dropna(axis=0, thresh=8)

df.head()

,cam_110,cam_111,cam_112,cam_114,cam_115,cam_116,cam_117,cam_118,cam_119,cam_120,cam_121,cam_122,cam_123,cam_219
2017-08-13 05:52:00,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,NaN,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,NaN,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...
2017-08-13 05:53:00,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,NaN,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,NaN,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...
2017-08-13 05:54:00,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,NaN,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,NaN,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...
2017-08-13 05:55:00,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,NaN,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,NaN,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...
2017-08-13 05:56:00,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,NaN,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,NaN,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...,/VISL2_net/amitibo/experiment_forecast_2017_08...


## Select a Specific Day

In [4]:
day = 21

df = df[(df.index>datetime.datetime(2017, 8, day)) & (df.index<datetime.datetime(2017, 8, day+1))]

##  Make the Movie

In [5]:
IMG_SIZE = 1200

In [6]:
def fixImage(img, gamma=1):
    img = img.astype(np.float)**gamma
    return (img/img.max()*255).astype(np.uint8)

In [7]:
video = cv2.VideoWriter(
    'solar_forcast.avi',
    cv2.VideoWriter_fourcc('X','2','6','4'),
    6,
    (IMG_SIZE, IMG_SIZE)
)

font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", 28, encoding="unic")

for _, row in tqdm_notebook(df.iterrows(), total=len(df)):
    black_img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
    for i, server_id in enumerate(row.index):
        if not type(row[server_id]) == str:
            #
            # nan camera
            #
            continue

        m = i // 4
        n = i - m * 4
        img = Image.open(row[server_id])
        if server_id != "cam_219":
            img = img.transpose(Image.FLIP_TOP_BOTTOM)
        draw = ImageDraw.Draw(img)
        draw.text(
            (0, 0),
            server_id,
            (255, 255, 255),
            font=font
        )    
        img = np.array(img)
        if server_id != "cam_219":
            img = fixImage(img, gamma=0.9)
        black_img[300*m:300*(m+1), 300*n:300*n+img.shape[1], ...] = img
    video.write(cv2.cvtColor(np.array(black_img), cv2.COLOR_RGB2BGR))

video.release()

A Jupyter Widget